# Example Query for Survival Probability of 1-hop Queries

Queries our system in the form of:
$P(survival\_time > 1000 | Mut\_g_1 = True$ $OR$ $Drug = d_1)$<br>
Returned is a knowledge graph contained probability of survival time.

In [1]:
import requests
import json
import csv

In [2]:
# Function: buildQuery
#
# Input:
# -----------
# list of gene tuples containing gene name and ENSEMBL Identifiers
#
# Output
# -----------
# A knowledge graph with decorated edges for 
# P(survival_time > 1000 | Mut_g1 = True OR Drug = d1)

def buildQuery(gene=None, drug=None):
    assert gene != None or drug != None, "Must include at least 1 gene OR 1 drug."
    
    # empty response
    reasoner_std = { "query_graph": dict()}
    
    # empty query graph
    reasoner_std["query_graph"] = { "edges": [],
                                    "nodes": []
                                  }
        
    if gene != None:
        # add in evidence genes
        reasoner_std['query_graph']['nodes'].append({ 'id':'n{}'.format('0'),
                                                      'type':'Gene',
                                                      'curie':'{}'.format(gene[1])
                                                   })
    elif drug != None:
        # add in evidence drug
        reasoner_std['query_graph']['nodes'].append({ 'id':'n{}'.format('0'),
                                                      'type':'Drug',
                                                      'name':'{}'.format(drug[0]),
                                                      'curie':'{}'.format(drug[1])
                                                   })

    # add target survival node
    reasoner_std['query_graph']['nodes'].append({ 'id': 'n{}'.format('1'),
                                                  'type': 'PhenotypicFeature',
                                                  'curie': 'UBERON:0000071',
                                                })
    # link evidence to target survival node
    reasoner_std['query_graph']['edges'].append({ 'id':'e{}'.format('0'),
                                                  'type':'causes',
                                                  'source_id':'n{}'.format('0'),
                                                  'target_id':'n{}'.format('1')
                                               })
    return reasoner_std

In [3]:
def readGenes():
    with open('gene_curie_map.csv', 'r') as gene_file:
        reader = csv.reader(gene_file)
        next(reader)
        rows = [row for row in reader]
    return rows

In [4]:
def readDrugs():
    with open('drug_curie_map.csv', 'r') as drug_file:
        reader = csv.reader(drug_file)
        next(reader)
        rows = [row for row in reader]
    return rows

In [5]:
# list of genes we can query over
#gene_list = readGenes()

# list of genes we can query over
#drug_list = readDrugs()

# or pull from above lists
gene = ('RAF1', 'ENSEMBL:ENSG00000132155')

query = buildQuery(gene=gene)
query['reasoner_id'] = 'ranking'
payload = {'query': query}

r = requests.post('http://chp.thayer.dartmouth.edu/submitQuery/', json=payload)
chp_res = json.loads(r.content)

QG = chp_res['query_graph']
KG = chp_res['knowledge_graph']
res = chp_res['results']

# extract probability
KG_result_edge = res['edge_bindings'][0]['kg_id']
for edge in KG['edges']:
    if edge['id'] == KG_result_edge:
        p_survival = edge['has_confidence_level']

# probability of surival given QG specification
print("Probability of survival > 1000 days is:", p_survival)

Probability of survival > 1000 days is: 0.42857142857142855
